# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [105]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [106]:
weather_data = "Output_Data/new_cities.csv"

weather_df = pd.read_csv(weather_data)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bredasdorp,-34.53,20.04,59.00,87,25,4.70,ZA,1609633639
1,Nichinan,31.60,131.37,44.60,45,20,10.29,JP,1609633640
2,Chokurdakh,70.63,147.92,-29.36,86,26,5.73,RU,1609633640
3,Khatanga,71.97,102.50,-23.31,83,41,9.64,RU,1609633443
4,Punta Arenas,-53.15,-70.92,51.80,62,0,23.04,CL,1609633640


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [107]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df['Humidity'].astype(float)

fig = gmaps.figure(center=(10, 0.1), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df['Humidity']),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [108]:
ideal_weather = weather_df.loc[(weather_df['Max Temp'] > 60) & (weather_df['Max Temp'] < 70) & (weather_df['Wind Speed'] < 5) & (weather_df['Cloudiness'] < 10)]
hotel_df = ideal_weather.dropna()

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,Hermanus,-34.42,19.23,64.99,85,2,3.00,ZA,1609633644
45,Salalah,17.02,54.09,69.80,73,0,2.24,OM,1609633650
93,Port Alfred,-33.59,26.89,64.00,89,5,4.00,ZA,1609633662
150,Kruisfontein,-34.00,24.73,60.01,90,0,4.47,ZA,1609633469
193,Port Elizabeth,-33.92,25.57,66.20,88,0,3.36,ZA,1609633685
550,Matamoros,25.53,-103.25,60.80,31,5,3.33,MX,1609633772


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [109]:
hotels = [''] * len(hotel_df.index)
hotel_df.insert(9, 'Hotel Name', hotels, True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,Hermanus,-34.42,19.23,64.99,85,2,3.00,ZA,1609633644,
45,Salalah,17.02,54.09,69.80,73,0,2.24,OM,1609633650,
93,Port Alfred,-33.59,26.89,64.00,89,5,4.00,ZA,1609633662,
150,Kruisfontein,-34.00,24.73,60.01,90,0,4.47,ZA,1609633469,
193,Port Elizabeth,-33.92,25.57,66.20,88,0,3.36,ZA,1609633685,
550,Matamoros,25.53,-103.25,60.80,31,5,3.33,MX,1609633772,


In [110]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': 5000,
    "type": "lodging",
    "key": g_key,
}

hotel_coord = []
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    lat = row['Lat']
    lon = row['Lng']
    city = row['City']

    # add keyword to params dict
    params['location'] = f'{lat},{lon}'

    # assemble url and make API request
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_coord.append((response['results'][0]['geometry']['location']['lat'], response['results'][0]['geometry']['location']['lng']))
    
    except:
        print(f'No hotel within 5000 meters of {city}')
        hotel_df.loc[index, 'Hotel Name'] = "None within 5000 meters"


In [111]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,Hermanus,-34.42,19.23,64.99,85,2,3.00,ZA,1609633644,Misty Waves Boutique Hotel
45,Salalah,17.02,54.09,69.80,73,0,2.24,OM,1609633650,HAMDAN PLAZA HOTEL
93,Port Alfred,-33.59,26.89,64.00,89,5,4.00,ZA,1609633662,The Halyards Hotel
150,Kruisfontein,-34.00,24.73,60.01,90,0,4.47,ZA,1609633469,Oyster Bay House Rental
193,Port Elizabeth,-33.92,25.57,66.20,88,0,3.36,ZA,1609633685,39 On Nile Guest House
550,Matamoros,25.53,-103.25,60.80,31,5,3.33,MX,1609633772,Recreativo Sabdy Briseidy


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [114]:
# Assign the marker layer to a variable.
# Info box content code found https://jupyter-gmaps.readthedocs.io/en/latest/api.html

markers = gmaps.marker_layer(hotel_coord, info_box_content = hotel_info)

# Add the layer to the map

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))